In [42]:
import pyglet.gl as gl
import numpy as np
import pyrealsense2 as rs
import open3d as o3d
import matplotlib.pyplot as plt     
from datetime import datetime
%matplotlib inline
import PyQt5
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import plotly.graph_objects as go

In [43]:
class PointObject:
    def __init__(self, point_cloud,depth,rgb,timestamp):
        self.point_cloud = point_cloud
        self.depth = depth
        self.rgb = rgb
        self.timestamp = timestamp

In [70]:
def depth_cam_collection():
    point_cloud_list = []

    # Declare pointcloud object, for calculating pointclouds and texture mappings
    pc = rs.pointcloud()
    # We want the points object to be persistent so we can display the last cloud when a frame drops
    points = rs.points()

    # Declare RealSense pipeline, encapsulating the actual device and sensors
    pipe = rs.pipeline()
    config = rs.config()

    # Enable depth and colour stream
    config.enable_stream(rs.stream.depth)
    config.enable_stream(rs.stream.color)

    # Start streaming with chosen configuration
    pipe.start(config)

    # We'll use the colorizer to generate texture for our PLY
    # (alternatively, texture can be obtained from color or infrared stream)
    colorizer = rs.colorizer()

    try:
        # Skip 5 first frames to give the Auto-Exposure time to adjust
        for x in range(10):
            pipe.wait_for_frames()

        while True:
            # Wait for the next set of frames from the camera
            frames = pipe.wait_for_frames()
            colorized = colorizer.process(frames)

            # Check the colour image
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()
            time_stamp = datetime.now()
            print(time_stamp)
            
            color = np.asanyarray(color_frame.get_data())
            depth = np.asanyarray(depth_frame.get_data())

            # fig, (ax1, ax2) = plt.subplots(1, 2)
            # fig.suptitle('Horizontally stacked subplots')
            # ax1.imshow(color)
            # ax2.imshow(depth)

            # Create save_to_ply object
            ply = rs.save_to_ply("1.ply")
            # Set options to the desired values
            # In this example we'll generate a textual PLY with normals (mesh is already created by default)
            ply.set_option(rs.save_to_ply.option_ply_binary, False)
            ply.set_option(rs.save_to_ply.option_ply_normals, True)

            print("Saving to 1.ply...")
            # Apply the processing block to the frameset which contains the depth frame and the texture
            ply.process(colorized)
            print("Done")
            pcd = o3d.io.read_point_cloud("1.ply") # Load saved pointcloud
            point_cloud_data  = np.asarray(pcd.points) # Check the issue with pickle

            downpcd = pcd.voxel_down_sample(voxel_size=0.01)
            down_point_cloud_data  = np.asarray(downpcd.points)# Check the issue with pickle
            real_cloud = downpcd
            #print("Downsized voxel data")
            #print(down_point_cloud_data.shape)

            #Remove radius outlier
            #down_outlier_removed,_= downpcd.remove_radius_outlier(100,0.5)
            # #o3d.visualization.draw_geometries([downpcd])
            #o3d.visualization.draw_geometries([down_outlier_removed])

            distance = 0.5
            p0 = np.array([0,0,0])
            masked_list = []
            for i in down_point_cloud_data:
                dist = np.linalg.norm(i - p0)
                if dist < distance:
                    masked_list.append(i)
            print(len(masked_list)) 
            distance_point_cloud_data = np.asarray(masked_list)
            print("Distanced Point cloud data")
            print(distance_point_cloud_data.shape)

            pcd_down = o3d.geometry.PointCloud()
            pcd_down.points = o3d.utility.Vector3dVector(distance_point_cloud_data)
            #pcd_down.lookat  = pcd.lookat
            o3d.visualization.draw_geometries([downpcd])
            o3d.visualization.draw_geometries([pcd_down])

            #fig = plt.figure(figsize=(20,10))

            # fig = plt.figure()
            # ax4 = Axes3D(fig)

            #ax1 = fig.add_subplot(111, projection='3d')
            #ax2 = fig.add_subplot(122)
            #ax3 = fig.add_subplot(133)

            #ax4 = fig.add_subplot(122,projection='3d')
            #ax4 = fig.add_subplot(111,projection='3d')
            #ax5 = fig.add_subplot(222)
            #ax6 = fig.add_subplot(233)

            #down_point_cloud_data = down_point_cloud_data.T
            # ax1.set_title("3D Scatter Plot (Voxelized)")
            # ax1.scatter(down_point_cloud_data[0], down_point_cloud_data[1], down_point_cloud_data[2])
            # ax1.set_xlabel('Azimuth(m)')
            # ax1.set_ylabel('Range(m)')
            # ax1.set_zlabel('Elevation(m)')
            # ax1.set_xlim(left=-4, right=4)
            # ax1.set_ylim(bottom=0, top=8)
            # ax1.set_zlim(top=-4, bottom=4)
            # ax1.grid(b=True)

            # ax2.set_title("Range Azimuth Plot")
            # ax2.set_ylim(bottom=0, top=8)
            # ax2.set_ylabel('Range(m)')
            # ax2.set_xlim(left=-4, right=4)
            # ax2.set_xlabel('Azimuth(m)')
            # ax2.grid(b=True)
            # ax2.scatter(down_point_cloud_data[0], down_point_cloud_data[1])

            # ax3.set_title("Range Elevation Plot")
            # ax3.set_ylim(bottom=0, top=8)
            # ax3.set_ylabel('Range(m)')
            # ax3.set_xlim(left=-4, right=4)
            # ax3.set_xlabel('Elevation(m)')
            # ax3.grid(b=True)
            # ax3.scatter(down_point_cloud_data[0], down_point_cloud_data[2])

            # distance_point_cloud_data = distance_point_cloud_data.T
            # ax4.set_title("3D Scatter Plot (Distance Reduction)")
            # ax4.plot_trisurf(distance_point_cloud_data[0], distance_point_cloud_data[1], distance_point_cloud_data[2])
            # ax4.set_xlabel('Azimuth(m)')
            # ax4.set_ylabel('Range(m)')
            # ax4.set_zlabel('Elevation(m)')
            # ax4.set_xlim(left=-2, right=2)
            # ax4.set_ylim(bottom=0, top=3)
            # ax4.set_zlim(top=-2, bottom=2)
            # ax4.grid(b=True)

            # fig = px.mesh_3d(
            #     x = distance_point_cloud_data[0], 
            #     y = distance_point_cloud_data[1], 
            #     z = distance_point_cloud_data[2],
            #     size_max = 1,
            #     range_x =(0,2),
            #     range_y =(-1,1),
            #     range_z =(-1,1)
            #     )

            # fig.show()

            # fig = go.Figure(data=[go.Surface(
            #  x=distance_point_cloud_data[0],
            #  y=distance_point_cloud_data[1], 
            #  z=distance_point_cloud_data[2])
            #  ])

            # fig.update_layout(title='Mt Bruno Elevation', autosize=False,
            #       width=500, height=500,
            #       margin=dict(l=65, r=50, b=65, t=90))
            # fig.show()


            # fig.update_layout(
            #         scene = dict(
            #          xaxis = dict(nticks=4, range=[0,2],),
            #          yaxis = dict(nticks=4, range=[-1,1],),
            #          zaxis = dict(nticks=4, range=[-1,1],),),
            #          )
            


            #plt.subplots_adjust(bottom=0.15, wspace=3)
            #fig.tight_layout()
            #fig.subplots_adjust(wspace=2)
            #plt.show()

            #Selecting first 1024 items
            # masked_list = masked_list[:1024]
            # point_cloud_data = np.asarray(masked_list).T 
            # print(point_cloud_data.shape)

            # reshaped = np.reshape(down_point_cloud_data,(-1,32,32))
            # print(reshaped.shape)
            # point_object = PointObject(reshaped,depth,color,time_stamp)
            # point_cloud_list.append(point_object)
            break;
        
    finally:
        pipe.stop()
        arr = np.asarray(point_cloud_list)
        np.save("depth_cam.npy",arr,allow_pickle=True)
    return distance_point_cloud_data,real_cloud

In [71]:
down_point_cloud_data = None
down_point_cloud_data,original = depth_cam_collection()

2021-08-11 22:19:40.736256
Saving to 1.ply...
Done
964
Distanced Point cloud data
(964, 3)
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [52]:
pcd_down = o3d.geometry.PointCloud()
pcd_down.points = o3d.utility.Vector3dVector(down_point_cloud_data)
o3d.visualization.draw_geometries([pcd_down])


In [72]:
%matplotlib qt
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(111, projection='3d')
distance_point_cloud_data = down_point_cloud_data.T
ax1.set_title("3D Scatter Plot (Distance Reduction)")
im = ax1.scatter(
    distance_point_cloud_data[0], 
    distance_point_cloud_data[2]*-1, 
    distance_point_cloud_data[1],
    s = 10,
    alpha = 0.5,
    c = distance_point_cloud_data[0],
    marker = ".",
    cmap = "winter"
    )
ax1.set_xlabel('Azimuth-x')
ax1.set_ylabel('Range - z')
ax1.set_zlabel('Elevation - y')
# ax1.set_xlim(left=-0.5, right=0.5)
# ax1.set_ylim(bottom=0, top=1)
# ax1.set_zlim(top=-0.5, bottom=0.5)
ax1.grid(b=True)
fig.colorbar(im, ax=ax1)


In [48]:
distance_point_cloud_data.shape

(3, 19738)